# ニュース一覧から、ニュースの本文に書かれている文字を取得

ここでは、先程取得した、ニュース一覧のテキストファイルから、URLを抜き出し、そのURLのページに書かれている本文を取得しましょう。

本文の文字列のみを取得し、ファイルに書き込みます。その際に、ページの区切りを "===" という行を追加して、一つの `今日の日付-news-text.txt` ファイルに保存しましょう。

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date

In [2]:
today = date.today()
list_filename = f"{today:%Y%m%d}-news-list.txt"

In [3]:
# ColabでPathがうまく取れない場合は次のセルを参照
links = []
with open(list_filename, "r", encoding="utf-8") as f:
    for line in f:
        title, link, date_ = line.split(",")
        links.append(link)

links[0]

'http://www.hit-u.ac.jp/news/10531'

In [4]:
# Colabでファイルが取得できない場合
import csv
import urllib.request
import base64
import io
url = "https://hit-u-data-text-processing.herokuapp.com/data/20210612-news-list.txt"
auth_str = base64.b64encode(b"reader:hit-u")
req = urllib.request.Request(url,
                            headers={"Authorization": "Basic " + auth_str.decode("utf-8")})

links = []
with urllib.request.urlopen(req) as req:
    news_data = req.read().decode("utf-8")
    with io.StringIO(news_data) as f:        
        for line in f:
            title, link, date_ = line.split(",")
            links.append(link)

links[0]

'https://www.hit-u.ac.jp/extramural/kansai-a/report/18th_20210227.html'

In [5]:
out = []
for link in links:
    r = requests.get(link)
    r.encoding = "utf-8"
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    try:
        text = soup.find(id="main").text
    except AttributeError as e:
        print(f"No main id, url is {link}")
    else:
        out.append(text)

out[0][:200]

No main id, url is https://www.sba.hub.hit-u.ac.jp/hitmagazine/2021/05/3030.html
No main id, url is http://www.hit-u.ac.jp/1284wp/wp-content/uploads/2021/02/hit-u_econ_20210210.pdf


'\n関西・中部合同アカデミア\n2020年度一橋大学関西・中部合同アカデミア\n  「新局面に入る米中の戦略的競争と日本」\n\n\u30002021年２月27日（土）、「新局面に入る米中の戦略的競争と日本」をテーマに、2020年度関西・中部合同アカデミアをオンラインセミナーとして開催しました。\n\u3000プログラムは、秋山信将法学研究科／国際・公共政策大学院教授の進行のもと、大月康弘副学長による開会挨拶で始まり、中山俊宏氏'

In [6]:
text_filename = f"{today:%Y%m%d}-news-text.txt"

In [7]:
with open(text_filename, "w", encoding="utf-8") as f:
    for text in out:
        f.write(text)
        f.write("\n===\n")